## Problema de la dieta 

Santigo Casasbuenas - 202214932

Amalia Carbonell - 202122079 

**1. Definicion de conjuntos y parametros:** 

$J$ : es el conjunto de alimentos $J =  \{1,2,..., m\}$

$N$ : es el conjunto de nutrientes (proteina, grasas, azucares carbohidratos y calorias) $N =  \{1,2,..., n\}$

- (j ∈ J)(j recorre alimentos )

- (i ∈ N)( i recorre nutrientes )

Para cada alimento i ∈ J y nutriente i ∈ N se define: 
- $a_{ij}$ --> cantidad de nutrientes $i$ en el alimento $j$
- $c_{j}$ --> el costo del alimento $j$

Para cada nutriente i ∈ N definimos 
- $L_{i}$ --> Limite minimo recomendado para el nutriente $i$
- $V_{i}$ --> Limite maximo permitido para el nutriente $i$

**2. Variable de decision** 
$X_{j} \geq 0$ : cantidad de porciones del alimento j que se van a consumir 
 
**3. Funcion objetivo**
-   minimicar el costo total de la dieta. ( Suma del costo de cada alimento * las porciones que se van a consumir)
-   $$\min \sum_{j \in J} c_j x_j$$

**4. Restricciones:** 

Para cada nutriente $i ∈ N$ se debe cumplir: 

1. Consumo minimo (para proteinas y calorias)
$$sum_{j \in J} a_{ij} x_j \geq L_i$$

2. Consumo Maximo 
$$sum_{j \in J} a_{ij} x_j \leq V_i$$

In [1]:
# ----------------------------------------------------------------------------
# MODELO GENERALIZADO DE DIETA EN PYOMO


from pyomo.environ import *


# 1. DEFINIMOS LOS DATOS DE FORMA "GENERALIZADA"
# Conjunto de alimentos
Foods = ['Carne', 'Arroz', 'Leche', 'Pan']

# Costo por porción (en la unidad de porción definida)
cost_data = {
    'Carne': 3000,  # COP
    'Arroz': 1000,
    'Leche':  600,
    'Pan':    700
}

# Conjunto de nutrientes
Nutrients = ['proteinas', 'grasas', 'azucares', 'carbohidratos', 'calorias']

# Contenido de cada nutriente n en el alimento f
content_data = {
    ('proteinas', 'Carne'): 26.0,
    ('proteinas', 'Arroz'): 4.2,
    ('proteinas', 'Leche'): 8.0,
    ('proteinas', 'Pan'):   6.0,

    ('grasas', 'Carne'): 19.3,
    ('grasas', 'Arroz'): 0.5,
    ('grasas', 'Leche'): 8.0,
    ('grasas', 'Pan'):   0.8,

    ('azucares', 'Carne'): 0.0,
    ('azucares', 'Arroz'): 0.01,
    ('azucares', 'Leche'): 13.0,
    ('azucares', 'Pan'):   25.0,

    ('carbohidratos', 'Carne'): 0.0,
    ('carbohidratos', 'Arroz'): 44.1,
    ('carbohidratos', 'Leche'): 11.0,
    ('carbohidratos', 'Pan'):   55.0,

    ('calorias', 'Carne'): 287.0,
    ('calorias', 'Arroz'): 204.0,
    ('calorias', 'Leche'): 146.0,
    ('calorias', 'Pan'):   245.0,
}

# Requisitos mínimos (None si no aplica)
min_req_data = {
    'proteinas': 63.0,   # >= 63 g
    'grasas':    None,   # sin mínimo
    'azucares':  None,   # sin mínimo
    'carbohidratos': None,
    'calorias':  1500.0  # >= 1500 Cal
}

# Requisitos máximos (None si no aplica)
max_req_data = {
    'proteinas': None,  # sin máximo
    'grasas':    50.0,  # <= 50 g
    'azucares':  25.0,  # <= 25 g
    'carbohidratos': 200.0,  # <= 200 g
    'calorias':  None   # sin máximo
}

# 2. CONSTRUIMOS EL MODELO EN PYOMO
model = ConcreteModel(name="Modelo Generalizado de Dieta")

# 2.1 Conjuntos

model.F = Set(initialize=Foods, doc="Conjunto de alimentos")
model.N = Set(initialize=Nutrients, doc="Conjunto de nutrientes")

# 2.2 Parámetros

model.cost = Param(model.F, initialize=cost_data, doc="Costo por porción")
def content_init(model, n, f):
    return content_data[(n, f)]
model.content = Param(model.N, model.F, initialize=content_init,
                      doc="Cantidad del nutriente n en alimento f")

# Para manejar los límites mínimo y máximo, usamos un truco:
# Si min_req_data[n] (o max_req_data[n]) es None, establecemos ±infinito.
BIGM = 1e8  # un número muy grande
def min_req_init(model, n):
    return min_req_data[n] if min_req_data[n] is not None else 0.0
def max_req_init(model, n):
    return max_req_data[n] if max_req_data[n] is not None else BIGM

model.min_req = Param(model.N, initialize=min_req_init, doc="Requisito mínimo")
model.max_req = Param(model.N, initialize=max_req_init, doc="Requisito máximo")

# Para saber si un nutriente exige una cota inferior o superior real,
# podemos usar otra estructura, o simplemente comprobamos en las restricciones.


# 2.3 Variables de decisión

model.x = Var(model.F, domain=NonNegativeReals, doc="Número de porciones de cada alimento")


# 2.4 Función objetivo: Minimizar costo total
def obj_rule(m):
    return sum(m.cost[f] * m.x[f] for f in m.F)

model.cost_obj = Objective(rule=obj_rule, sense=1)  # sense=1 => Minimizar


# 2.5 Restricciones nutricionales


# Para cada nutriente n, verificamos si hay un mínimo y/o máximo “activo”.
# Si min_req_data[n] no es None, creamos restricción de >= min_req.
# Si max_req_data[n] no es None, creamos restricción de <= max_req.

def nutrient_min_rule(m, n):
    # Solo aplicamos si min_req_data[n] != None
    # (Aunque lo traducimos a 0 si era None, revisamos si min_req_data[n] no era None en su origen.)
    if min_req_data[n] is not None:
        return sum(m.content[n, f]*m.x[f] for f in m.F) >= m.min_req[n]
    else:
        # Sin mínimo (devolvemos Constraint.Skip)
        return Constraint.Skip

def nutrient_max_rule(m, n):
    # Solo aplicamos si max_req_data[n] != None
    if max_req_data[n] is not None:
        return sum(m.content[n, f]*m.x[f] for f in m.F) <= m.max_req[n]
    else:
        # Sin máximo
        return Constraint.Skip

model.min_constraints = Constraint(model.N, rule=nutrient_min_rule)
model.max_constraints = Constraint(model.N, rule=nutrient_max_rule)

# 3. RESOLVER EL MODELO

solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)


# 4. MOSTRAR RESULTADOS

print("\n=== RESULTADOS ===")
print("Estado del solver:", results.solver.status)
print("Óptimo encontrado:", results.solver.termination_condition)

print(f"\nCosto mínimo de la dieta: {value(model.cost_obj):.2f} COP\n")
for f in model.F:
    print(f"{f}: {model.x[f].value:.3f} porciones")

print("\nVerificación de nutrientes:")
for n in model.N:
    ingesta = sum(content_data[(n, f)] * model.x[f].value for f in model.F)
    # Si min_req_data[n] existe, chequear
    if min_req_data[n] is not None:
        print(f"{n} mínimo requerido = {min_req_data[n]:.2f}, ingerido = {ingesta:.2f}")
    # Si max_req_data[n] existe, chequear
    if max_req_data[n] is not None:
        print(f"{n} máximo permitido = {max_req_data[n]:.2f}, ingerido = {ingesta:.2f}")
    if min_req_data[n] is None and max_req_data[n] is None:
        print(f"{n} sin restricciones, ingerido = {ingesta:.2f}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/rx/2z7872sj27s0ywd3scyl9km80000gn/T/tmpmp9czjb5.glpk.raw
 --wglp /var/folders/rx/2z7872sj27s0ywd3scyl9km80000gn/T/tmprk_hw9av.glpk.glp
 --cpxlp /var/folders/rx/2z7872sj27s0ywd3scyl9km80000gn/T/tmpfiu3_7g8.pyomo.lp
Reading problem data from '/var/folders/rx/2z7872sj27s0ywd3scyl9km80000gn/T/tmpfiu3_7g8.pyomo.lp'...
5 rows, 4 columns, 18 non-zeros
50 lines were read
Writing problem data to '/var/folders/rx/2z7872sj27s0ywd3scyl9km80000gn/T/tmprk_hw9av.glpk.glp'...
39 lines were written
GLPK Simplex Optimizer 5.0
5 rows, 4 columns, 18 non-zeros
Preprocessing...
5 rows, 4 columns, 18 non-zeros
Scaling...
 A: min|aij| =  1.000e-02  max|aij| =  2.870e+02  ratio =  2.870e+04
GM: min|aij| =  1.177e-01  max|aij| =  8.497e+00  ratio =  7.219e+01
EQ: min|aij| =  1.385e-02  max|aij| =  1.000e+00  ratio =  7.219e+01
Constructing initial basis...
Size of triangular part is 5
      0: obj =   0.000000000e+0